В качестве входа нужны маски из [MIDV-500](https://arxiv.org/abs/1807.05786)
для предсказаний модель из 12 лекции: https://github.com/EPC-MSU/EduNet-secret/blob/dev-1.8/out/L12_Segmentation_Detection/EX12_Segmentation_sol.ipynb

# (Дополнительно) Сложно, долго, но интересно!

Делаем свой сканер документов по фотографии. Цель хорошо описывается картинками:

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/Exercises/EX12/Scaner.png"  width="900">

Если решились выполнять задание, то для удобства сохраните модель из предыдущего задания, а лучше некоторый набор предсказанных изображений: исходное, оригинальная маска, предсказанная маска. (весь validation датасет, например).

## Подзадача 1.
Используя маски, которые выдаёт модель, любыми алгоритмами найдите на масках углы документа (или границы документа).

Большинство известных алгоритмов можно найти в opencv, например:
* [Canny](https://docs.opencv.org/4.x/da/d22/tutorial_py_canny.html)
* [Contour features](https://docs.opencv.org/4.6.0/dd/d49/tutorial_py_contour_features.html)
* [Features harris](https://docs.opencv.org/4.x/dc/d0d/tutorial_py_features_harris.html)

Рекомендуется перед применением алгоритмов [маску бинаризовать](https://docs.opencv.org/3.4/db/d8e/tutorial_threshold.html) по порогу: все пиксели выше порога сделать истинно белыми, а ниже - чёрными. (И проверить качество бинаризованных изображений)

In [ ]:
import cv2 as cv

# cur_mask - numpy-array mask, values between 0 and 1
# returns binarized mask as uint8 numpy array
def binarize_mask(cur_mask, dilation_rad=5):
    cur_mask = (np.clip(cur_mask, 0, 1) * 255.0).astype(np.uint8)
    h, w = cur_mask.shape
    # plt.imshow(cur_mask)

    # Since ground truth values are 0 or 1 and network output may be outside of that,
    # setting a constant threshold for ground truth is workable
    _, cur_mask = cv.threshold(cur_mask, 128, 255, cv.THRESH_BINARY)
    padded_mask = np.zeros(
        (h + dilation_rad * 2 + 2, w + dilation_rad * 2 + 2), np.uint8
    )
    padded_mask[
        dilation_rad + 1 : -dilation_rad - 1, dilation_rad + 1 : -dilation_rad - 1
    ] = cur_mask

    structuringElem = cv.getStructuringElement(
        cv.MORPH_RECT, (dilation_rad, dilation_rad)
    )
    padded_mask = cv.dilate(padded_mask, structuringElem)

    # Filling the image
    # based on https://learnopencv.com/filling-holes-in-an-image-using-opencv-python-c/
    mask_floodfill = padded_mask.copy()
    mask = np.zeros((h + dilation_rad * 2 + 4, w + dilation_rad * 2 + 4), np.uint8)
    mask_floodfill = cv.floodFill(
        mask_floodfill, mask, (0, 0), 255, flags=cv.FLOODFILL_MASK_ONLY
    )[1]
    mask = 255 - 255 * mask
    mask = cv.erode(mask, structuringElem)
    # plt.imshow(cur_mask)
    return mask[
        dilation_rad + 2 : -dilation_rad - 2, dilation_rad + 2 : -dilation_rad - 2
    ]

In [ ]:
def find_corners(mask, dilation_size=3):
    # Expanding mask a bit
    structuringElem = cv.getStructuringElement(
        cv.MORPH_RECT, (dilation_size, dilation_size)
    )
    mask = cv.dilate(mask, structuringElem)

    contours = cv.findContours(mask, cv.RETR_LIST, cv.CHAIN_APPROX_SIMPLE)[0]

    # Finding the largest segment
    best_ind = 0
    best_area = cv.contourArea(contours[0])
    for i in range(1, len(contours)):
        cur_area = cv.contourArea(contours[i])
        if cur_area > best_area:
            best_area = cur_area
            best_ind = i

    contours = contours[best_ind]
    epsilon = 0.05 * cv.arcLength(contours, True)
    contours = cv.approxPolyDP(contours, epsilon, True)
    if len(contours) != 4:
        # If no good rectangle found, leave the image unchanged
        contours = np.array([[0, 0], [0, 223], [223, 223], [223, 0]])
    else:
        contours = np.array([point[0] for point in contours])

    return contours

In [ ]:
it = iter(valset)
batch = next(it)
# batch = next(it)
# batch = next(it)
img, mask = batch
output = model(img.unsqueeze(0).to(device)).cpu().squeeze()
# print(output.size())
cur_mask = output.detach().numpy()

cur_mask = binarize_mask(cur_mask)
# plt.imshow(cur_mask)
corners = find_corners(cur_mask)
print(cur_mask.shape)
corners

## Подзадача 2.

Произвести трансформацию перспективы по найденным углам/контурам. Разумеется, трансформацию необходимо производить над исходным изображением, а не над масками.


Для этого используются функции `cv2.getPerspectiveTransform` и
`cv2.warpPerspective`.

In [ ]:
import cv2 as cv

print(corners.astype(np.float32))
# perspTransfom = cv2.getPerspectiveTransform(corners.astype(np.float32), np.float32([[0,0], [0,223], [223,223], [223, 0]]))
perspTransfom = cv.getPerspectiveTransform(
    np.float32([[59, 125], [185, 130], [186, 177], [44, 170]]),
    np.float32([[0, 0], [223, 0], [223, 223], [0, 223]]),
)
print(perspTransfom)
result = cv.warpPerspective(
    np.transpose(img.detach().numpy(), (1, 2, 0)), perspTransfom, (224, 224)
)
plt.imshow(result)

In [ ]:
screenConv = np.float32([[0, 0], [223, 0], [223, 223], [0, 223]])

for batch in valset:
    img, mask = batch
    output = model(img.unsqueeze(0).to(device)).cpu().squeeze()
    output = output.detach().numpy()
    cur_mask = binarize_mask(output)
    img = np.transpose(img.detach().numpy(), (1, 2, 0))

    cornersRes = find_corners(cur_mask)
    perspTransfomRes = cv.getPerspectiveTransform(
        cornersRes.astype(np.float32), screenConv
    )
    convertedRes = cv.warpPerspective(img, perspTransfomRes, (224, 224))

    mask = np.transpose(mask.detach().numpy(), (1, 2, 0))
    mask = (mask * 255).astype(np.uint8)
    cornersOrig = find_corners(mask)
    perspTransfomOrig = cv.getPerspectiveTransform(
        cornersOrig.astype(np.float32), screenConv
    )
    convertedOrig = cv.warpPerspective(img, perspTransfomOrig, (224, 224))

    fig, ax = plt.subplots(nrows=2, ncols=3, figsize=(30, 20))
    ax[0, 0].imshow(img)
    ax[0, 0].set_title("Original")
    ax[0, 1].imshow(mask[:, :, 0])
    ax[0, 1].set_title("GT mask")
    ax[0, 2].imshow(convertedOrig)
    ax[0, 2].set_title("Conversion from GT mask")
    ax[1, 0].imshow(output)
    ax[1, 0].set_title("Network output")
    ax[1, 1].imshow(cur_mask)
    ax[1, 1].set_title("Processed output")
    ax[1, 2].imshow(convertedRes)
    ax[1, 2].set_title("Conversion from output mask")

    plt.show()

## Памятка для преподавателя
<...>